# SIADS 516: Homework 1
Version 1.0.20200221.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### Our first mrjob script

Recall the following example from the lectures:

Note the use of the magic command ```%%file```.  You can use this to write the contents of a cell out to a file, which is what we need to do to use mrjob:

In [1]:
%%file word_count.py
from mrjob.job import MRJob
import re

class MRWordFrequencyCount(MRJob):

  ### input: self, in_key, in_value
  def mapper(self, _, line):
    yield "chars", len(line)
    yield "words", len(line.split())
    yield "lines", 1

  ### input: self, in_key from mapper, in_value from mapper
  def reducer(self, key, values):
    yield key, sum(values)
if __name__ == "__main__":
    MRWordFrequencyCount.run()

Overwriting word_count.py


### <font color="magenta">Q1: Explain what each of the yield statements in the above script do.  Provide a list of what the first few iterations through the mapper() step would yield if the script was run against the ```data/gutenberg/short.t1.txt``` file.

The "chars" yield statement will return the length of each line in the file as a way of determining, eventually, the total number of characters in the txt file. "chars" will be the key that is created, The "words" yield statement will split the line on white space and return the length of the subsequently created list. Taking the length of the list will therefore return the number of words that were in the line. "words" will be the key that is created. The "line" yield statement is defaulted to 1 and therefore every line will return a 1 as part of this yield. This will eventually allow for the number of total lines to be determined. "lines" is the key that will be created.

One important note with these results is that the '\n' line break character will be included in the overall character count but will not count as a word. How the first ten lines of the file would process through the mapper step are seen below. 

In [2]:
lines = []
with open('data/gutenberg/short.t1.txt') as q1:
    lines = q1.readlines()

    for line in lines[0:10]:
        print("chars:", len(line), "words:", len(line.split()), "lines:", 1)

chars: 70 words: 12 lines: 1
chars: 1 words: 0 lines: 1
chars: 65 words: 14 lines: 1
chars: 69 words: 12 lines: 1
chars: 68 words: 11 lines: 1
chars: 47 words: 7 lines: 1
chars: 1 words: 0 lines: 1
chars: 1 words: 0 lines: 1
chars: 24 words: 5 lines: 1
chars: 1 words: 0 lines: 1


Now let's look at the output of running the script against that same file:

In [3]:
!python word_count.py data/gutenberg/short.t1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20210705.220500.899753
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20210705.220500.899753/output
Streaming final output from /tmp/word_count.jovyan.20210705.220500.899753/output...
"chars"	10653
"lines"	200
"words"	1822
Removing temp directory /tmp/word_count.jovyan.20210705.220500.899753...


### <font color="magenta">Q2.  Repeat the above cell using the the works of William Shakespeare text file (data/gutenberg/t8.shakespeare.txt).  Provide an interpretation of the output (don't overthink this -- just demonstrate that you can find the relevant information in the output).</font>

In [4]:
!python word_count.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20210705.220501.958480
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20210705.220501.958480/output
Streaming final output from /tmp/word_count.jovyan.20210705.220501.958480/output...
"lines"	124456
"words"	901325
"chars"	5333743
Removing temp directory /tmp/word_count.jovyan.20210705.220501.958480...


In terms of the relevant information from this output, the mapper and reducer functions show that, in the Shakespeare file, there are 901,325 words, 124,456 lines, and 5,333,743 characters.

### Now let's look at a slightly more complicated example:

In [5]:
%%file most_used_word.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord.run()
    end = time.time()
    print(end - start)

Overwriting most_used_word.py


### <font color="magenta">Q3: Explain what the yield statements in the  above script do.  Provide a list of what the first few iterations through the steps would yield.

The mapper_get_words yield statement will, for every word found by the regex pattern, create a key which is the lowercase of the word and assign a 1, denoting that the word was in the file. This mapping will occur for all words. The combiner_count_words will again return the word as the key but is summing the count of the words seen so far in the particular node, prior to going to the reducer. For the reducer_count_words yield statement, no key (or a key of None) is returned but a sum of counts for each word is yielded as a value pair to then move into the final reducing step. The final step, the reducer_find_max_word step, will take the pair created in the prior step and yield a key, which will be the max count, and will also yield the word itself as a value, which is tied to the max count.

In [6]:
#The First Step Output: mapper_get_words through the first 20 lines
import re
WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below
STOPWORDS = ['i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

lines1 = []
with open('data/gutenberg/short.t1.txt') as q31:
    lines1 = q31.readlines()

    for line in lines1[0:20]:
        for word in WORD_RE.findall(line):
            if word.lower() not in STOPWORDS:
                print(word.lower(), 1)

project 1
gutenberg's 1
year 1
2889 1
jules 1
verne 1
michel 1
verne 1
ebook 1
use 1
anyone 1
anywhere 1
cost 1
almost 1
restrictions 1
whatsoever 1
may 1
copy 1
give 1
away 1
re 1
use 1
terms 1
project 1
gutenberg 1
license 1
included 1
ebook 1
online 1
www 1
gutenberg 1
org 1
title 1
year 1
2889 1
author 1
jules 1
verne 1
michel 1
verne 1
release 1
date 1
january 1
2 1
2007 1
ebook 1
19362 1
original 1
version 1
posted 1
september 1
23 1
2006 1
language 1
english 1
character 1
set 1
encoding 1
ascii 1
start 1
project 1
gutenberg 1
ebook 1
year 1
2889 1


In [7]:
#The Second Step Output: combiner_count_words through the first 20 lines
lines2 = []
word_dict = {}
with open('data/gutenberg/short.t1.txt') as q32:
    lines2 = q32.readlines()

    for line in lines2[0:20]:
        for word in WORD_RE.findall(line):
            if word.lower() not in STOPWORDS:
                if word.lower() not in word_dict:
                    word_dict[word.lower()] = 1
                else:
                    word_dict[word.lower()] += 1

for word in word_dict:
    print(word, word_dict[word])

project 3
gutenberg's 1
year 3
2889 3
jules 2
verne 4
michel 2
ebook 4
use 2
anyone 1
anywhere 1
cost 1
almost 1
restrictions 1
whatsoever 1
may 1
copy 1
give 1
away 1
re 1
terms 1
gutenberg 3
license 1
included 1
online 1
www 1
org 1
title 1
author 1
release 1
date 1
january 1
2 1
2007 1
19362 1
original 1
version 1
posted 1
september 1
23 1
2006 1
language 1
english 1
character 1
set 1
encoding 1
ascii 1
start 1


In [8]:
#The Third Step Output: reducer_count_words through the first 20 lines
lines3 = []
word_dict2 = {}
with open('data/gutenberg/short.t1.txt') as q33:
    lines3 = q33.readlines()

    for line in lines3[0:20]:
        for word in WORD_RE.findall(line):
            if word.lower() not in STOPWORDS:
                if word.lower() not in word_dict2:
                    word_dict2[word.lower()] = 1
                else:
                    word_dict2[word.lower()] += 1

for word in word_dict2:
    pairs = [(value, key) for key, value in word_dict2.items()]

pairs

[(3, 'project'),
 (1, "gutenberg's"),
 (3, 'year'),
 (3, '2889'),
 (2, 'jules'),
 (4, 'verne'),
 (2, 'michel'),
 (4, 'ebook'),
 (2, 'use'),
 (1, 'anyone'),
 (1, 'anywhere'),
 (1, 'cost'),
 (1, 'almost'),
 (1, 'restrictions'),
 (1, 'whatsoever'),
 (1, 'may'),
 (1, 'copy'),
 (1, 'give'),
 (1, 'away'),
 (1, 're'),
 (1, 'terms'),
 (3, 'gutenberg'),
 (1, 'license'),
 (1, 'included'),
 (1, 'online'),
 (1, 'www'),
 (1, 'org'),
 (1, 'title'),
 (1, 'author'),
 (1, 'release'),
 (1, 'date'),
 (1, 'january'),
 (1, '2'),
 (1, '2007'),
 (1, '19362'),
 (1, 'original'),
 (1, 'version'),
 (1, 'posted'),
 (1, 'september'),
 (1, '23'),
 (1, '2006'),
 (1, 'language'),
 (1, 'english'),
 (1, 'character'),
 (1, 'set'),
 (1, 'encoding'),
 (1, 'ascii'),
 (1, 'start')]

In [9]:
#The Fourth Step Output: reducer_find_max_word through the first 20 lines
lines4 = []
word_dict3 = {}
with open('data/gutenberg/short.t1.txt') as q34:
    lines4 = q34.readlines()

    for line in lines4[0:20]:
        for word in WORD_RE.findall(line):
            if word.lower() not in STOPWORDS:
                if word.lower() not in word_dict3:
                    word_dict3[word.lower()] = 1
                else:
                    word_dict3[word.lower()] += 1

for word in word_dict3:
    pairs = [(value, key) for key, value in word_dict3.items()]

winning_val = str(max(pairs)[0])
winning_val2 = max(pairs)[1]
print(winning_val + " " + '"' + winning_val2 + '"')

4 "verne"


Now run the file against data/gutenberg/short.t1.txt.

In [10]:
!python most_used_word.py data/gutenberg/short.t1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20210705.220506.537753
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20210705.220506.537753/output
Streaming final output from /tmp/most_used_word.jovyan.20210705.220506.537753/output...
11	"day"
Removing temp directory /tmp/most_used_word.jovyan.20210705.220506.537753...
0.8281233310699463


### <font color="magenta">Q4: Run the above script on the Shakespeare text file.  What answer do you get?</font>

In [11]:
!python most_used_word.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20210705.220508.127528
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20210705.220508.127528/output
Streaming final output from /tmp/most_used_word.jovyan.20210705.220508.127528/output...
5479	"thou"
Removing temp directory /tmp/most_used_word.jovyan.20210705.220508.127528...
7.709667444229126


When running the above mrjob script on the Shakespeare text file it is found that it takes approximately 7.70 seconds to run and determine that "thou" is the most commonly used word in the file, being written 5,479 times.

### <font color="magenta">Q5: What is the impact of removing the combiner from the above code in terms of efficiency?  What does that suggest?</font>

In [12]:
%%file most_used_word_no_comb.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord_NoCombiner(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   #combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    #def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
     #   yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord_NoCombiner.run()
    end = time.time()
    print(end - start)

Overwriting most_used_word_no_comb.py


In [13]:
!python most_used_word_no_comb.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word_no_comb.jovyan.20210705.220516.644714
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word_no_comb.jovyan.20210705.220516.644714/output
Streaming final output from /tmp/most_used_word_no_comb.jovyan.20210705.220516.644714/output...
5479	"thou"
Removing temp directory /tmp/most_used_word_no_comb.jovyan.20210705.220516.644714...
5.910166025161743


When the combiner step was removed, this mrjob process ran in about 5.91 seconds on the Shakespeare text file, compared to the approximately 7.54 second run time when the combiner was included. The same result from the mrjob process was returned in each, with thou being the most commonly used word (5,479 times). This suggests that the combiner step is actually detrimental when this process is being run just on one notebook/machine, which is the case with this assignment. This additional step perhaps may be beneficial if there was the option to horizontally scale this process, but as shown based on run times, with one machine this pre-grouping prior to the reducing step only slows down the process and it is much more effective time-wise to map and then move directly to the reducing step.

### <font color="magenta">Q6: Write an mrjob script that finds the 10 words that have the most syllables from the t5.churchill.txt file.  Interpret your results.</font>

In [14]:
%%file most_syllables.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

#Syllable Counter Pulled from Stack Overflow
# https://stackoverflow.com/questions/46759492/syllable-count-in-python

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below

class MRMostSyllables(MRJob): 
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_syllables,
                reducer=self.reducer_syllable_pairs),
            MRStep(
                reducer=self.reducer_top_syllables
            )
        ]
        
    def mapper_syllables(self, _, line):
        for word in WORD_RE.findall(line):
            word = word.lower()
            count = 0
            vowels = "aeiouy"
            if word[0] in vowels:
                count += 1
            for index in range(1, len(word)):
                if word[index] in vowels and word[index - 1] not in vowels:
                    count += 1
            if word.endswith("e"):
                count -= 1
            if count == 0:
                count += 1 
            yield word.lower(), count
        
    def reducer_syllable_pairs(self, key, values):
        yield None, (max(values), key)
        
    def reducer_top_syllables(self, _, values):
        top_syllables = list(values)
        top_syllables.sort(reverse=True)

        top_10 = top_syllables[:10]
        for syllables, word in top_10:
            yield word, syllables


if __name__ == '__main__':
    import time
    start = time.time()
    MRMostSyllables.run()
    end = time.time()
    print(end - start)

Overwriting most_syllables.py


In [15]:
!python most_syllables.py data/gutenberg/t5.churchill.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_syllables.jovyan.20210705.220523.358351
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_syllables.jovyan.20210705.220523.358351/output
Streaming final output from /tmp/most_syllables.jovyan.20210705.220523.358351/output...
"overcapitalization"	8
"incommunicability"	8
"unenforceability"	7
"overcapitalized"	7
"materialistically"	7
"invulnerability"	7
"interrogatively"	7
"infinitesimally"	7
"indissolubility"	7
"indispensability"	7
Removing temp directory /tmp/most_syllables.jovyan.20210705.220523.358351...
18.270990133285522


The mrjob script created above took approximately 18.3 seconds to generate a list of the 10 words that have the most syllables in the Churchill text file. While there were several ties for words with 7 syllables (and thus it could be possible for other runs of this script to pull in other words that perhaps have this number of syllables), the final output reveals that overcapitalization and incommunicability were the words with the most syllables (8) while unenforceability, overcapitalized, materialistically, invulnerability, interrogatively, infinitesimally, indissolubility, and indispensability each had 7 syllables.